In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import numpy as np
import keras
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

2023-04-13 20:31:24.152372: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[]


In [3]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [4]:
paths = m21.corpus.getComposer('bach')
paths = paths[:10]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = utils.load_dataset_lstm(paths, 64, mapping)

print(f"Songs encoded: {len(paths)}")

Found 10 songs in corpus.
Removed 0 bad songs
Remaining songs: 10
Songs encoded: 10


In [5]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/bach_lstm"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, tensorboard_callback=tensorboard_callback, epochs=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               568320    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
173/173 [==============================] - 27s 126ms/step - loss: 1.9152 - accuracy: 0.6409


In [13]:
model.save("PinkFloyd_soloMelodie_transformer")

INFO:tensorflow:Assets written to: /home/floskinner/mukkeBude/mukkeBude/model/preTrainedModels/Bach_polyphonie_transformer/assets


INFO:tensorflow:Assets written to: /home/floskinner/mukkeBude/mukkeBude/model/preTrainedModels/Bach_polyphonie_transformer/assets


'/home/floskinner/mukkeBude/mukkeBude/model/preTrainedModels/Bach_polyphonie_transformer'

In [7]:
# Show the logs
%tensorboard --logdir logs/bach_lstm

In [8]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _", max_length=500)    
new_song = utils.decode_songs_old(generated_song)

1/1 [==============================] - 0s 19ms/step


In [9]:
path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}

/lib/x86_64-linux-gnu/libjack.so.0
/lib/x86_64-linux-gnu/libnss3.so
QEventLoop: Cannot be used without QApplication
QEventLoop: Cannot be used without QApplication
17:00:49.646 | INFO  | main_thread | GlobalModule | onPreInit: log path: /home/deglasfl/.local/share/MuseScore/MuseScore4/logs/MuseScore_230412_170049.log
17:00:49.646 | INFO  | main_thread | GlobalModule | onPreInit: === Started MuseScore 4.0.1, build number 230121751 ===
17:00:49.656 | ERROR | main_thread | LanguagesService | setCurrentLanguage: Error loading translator "/tmp/.mount_musesclvpw4j/share/mscore4portable-4.0//locale/qt_de.qm" 
17:00:49.822 | INFO  | main_thread | DiagnosticsModule | onInit: success start crash handler
17:00:49.884 | WARN  | main_thread | Qt         | QIODevice::read (QFile, "/home/deglasfl/.local/share/MuseScore/MuseScore4/shortcuts.xml"): device not open
17:00:49.884 | WARN  | main_thread | Qt         | QIODevice::read (QFile, "/home/deglasfl/.local/share/MuseScore/MuseScore4/midi_mappings.xm

# Train with new Dataformat from chinese dude

In [5]:
# Load songs
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

Found 10 songs in corpus.


/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:298: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:299: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,


In [6]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/bach_lstm_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=5, tensorboard_callback=tensorboard_callback)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               568320    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
113/113 [==============================] - 4s 15ms/step - loss: 3.1018 - accuracy: 0.2236
Epoch 2/5
113/113 [==============================] - 2s 15ms/step - loss: 2.5625 - accu

In [ ]:
# Save model
model.save("lstm_bach_polyphonie")

In [7]:
# Show the logs
%tensorboard --logdir logs/bach_lstm_polyphonie

In [ ]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep")    

new_song_ints = mapping.numericalize(generated_song)
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=100)

path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)
